In [1]:
import requests
import json
import requests
import pandas as pd
from plotly import graph_objects as go
import plotly.graph_objects as go
import numpy as np
import plotly.express as px
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
%run trips_duration.ipynb

object


/var/folders/6f/26l7p7y90rd4sw3bc17185400000gn/T/ipykernel_714/1627526132.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [3]:
def merging_data(final_1plus_df):

    tourist_num = final_1plus_df
    tourist_num_copy = final_1plus_df.copy()

    min_trips = tourist_num['NumTrips'].min()
    max_trips = tourist_num['NumTrips'].max()

    tourist_num['Tourism St.'] = (tourist_num['NumTrips'] - min_trips) / (max_trips - min_trips)

    tourist_num = pd.DataFrame(final_1plus_df, columns = ['Country', 'Year', 'Tourism St.'])
    tourist_num.sort_values(by = ["Year"])

    unfiltered_predicted_tourist =pd.DataFrame(tourist_num_copy, columns = ["Country", "Year", "NumTrips"])

    return tourist_num, unfiltered_predicted_tourist


In [4]:
datas = merging_data(final_1plus_df)
data = datas[0]
data_not_st = datas[1]
data

,Country,Year,Tourism St.
0,Albania,2017,0.014321
1,Albania,2019,0.018106
2,Albania,2020,0.011349
3,Austria,2012,0.067236
4,Austria,2013,0.068143
...,...,...,...
345,Switzerland,2018,0.076381
346,Switzerland,2019,0.069107
347,Switzerland,2020,0.059142
348,Switzerland,2021,0.061503


In [5]:
def get_lag_columns(country_df, lag = 3):

    country_df = country_df.sort_values('Year').reset_index(drop = True)
    country_df['Covid'] = country_df['Year'].apply(lambda x: 1 if x in [2020, 2021] else 0)
    country_df['Bias'] = 1
    
    index_country = country_df.index[country_df['Year'].astype(int) == 2015]
    index = index_country[0]

    y_vector = []
    x_matrix = []
    x_list = []
    for i in range(index, len(country_df)): 
        if i - lag < 0:
            continue
        bias = 1 
        country = country_df.iloc[i]["Country"]
        lag_values = [country_df.iloc[i - j]['Tourism St.'] for j in range(1, lag + 1)]
        covid = country_df.iloc[i]['Covid']
        year = country_df.iloc[i]['Year']
        
        x_list = [bias, country, year, covid] + lag_values
        x_matrix.append(x_list)
        y_vector.append(country_df.iloc[i]["Tourism St."])

    x_matrix = pd.DataFrame(x_matrix)
    x_matrix.rename(columns={0: "Bias", 1: "Country", 2:"Year", 3:"Covid", 4:"lag1", 5:"lag2", 6:"lag3"}, inplace=True)


    return x_matrix, y_vector

In [6]:
data[data['Country']=='Slovakia']

,Country,Year,Tourism St.
294,Slovakia,2012,0.021458
295,Slovakia,2013,0.020559
296,Slovakia,2014,0.021513
297,Slovakia,2015,0.025319
298,Slovakia,2016,0.030385
299,Slovakia,2017,0.034212
300,Slovakia,2018,0.039610
301,Slovakia,2019,0.044606
302,Slovakia,2020,0.019206
303,Slovakia,2021,0.023941


In [7]:
X, y = get_lag_columns(data[data['Country'] == 'Slovakia'], lag=3)
print(X)
print(y)

   Bias   Country  Year  Covid      lag1      lag2      lag3
0     1  Slovakia  2015      0  0.021513  0.020559  0.021458
1     1  Slovakia  2016      0  0.025319  0.021513  0.020559
2     1  Slovakia  2017      0  0.030385  0.025319  0.021513
3     1  Slovakia  2018      0  0.034212  0.030385  0.025319
4     1  Slovakia  2019      0  0.039610  0.034212  0.030385
5     1  Slovakia  2020      1  0.044606  0.039610  0.034212
6     1  Slovakia  2021      1  0.019206  0.044606  0.039610
7     1  Slovakia  2022      0  0.023941  0.019206  0.044606
8     1  Slovakia  2023      0  0.041598  0.023941  0.019206
[0.02531901720004173, 0.03038532634637201, 0.03421163044933105, 0.0396101720567253, 0.04460578041262051, 0.019206153354248967, 0.023941342883577185, 0.04159770013749046, 0.04807679240371502]


In [8]:
def fit_regression(X, y):
    Xt = X.T
    XtX = np.dot(Xt, X)
    XtX_inv = np.linalg.inv(XtX)
    Xt_y = np.dot(Xt, y)
    beta = np.dot(XtX_inv, Xt_y)

    return beta

In [9]:
def predict_next_year(beta, X):
    return np.dot(beta, X)

In [19]:
def get_country_years_pred(data, country, lag = 3):

    country_df = data[data['Country'] == country]
    country_df, y = get_lag_columns(country_df, lag = lag)

    lag_cols = [f'lag{i}' for i in range(1, lag + 1)]
    feature_cols = ['Bias', 'Covid'] + lag_cols
    
    X = country_df[feature_cols].values
    #print(X)
    #print(y)

    beta_tourism = fit_regression(X, y)
    #print(beta_tourism)

    predictions = []
    for index, row in country_df.iterrows():
        year = row['Year']
        X_year = np.array([row[col] for col in feature_cols])
        print(X_year)
        pred_tourism = predict_next_year(beta_tourism, X_year)

        predictions.append({
            'Country': country,
            'Year': year,
            'Actual_Tourism': y[index],
            'Predicted_Tourism': pred_tourism

        })

    all_years_results = pd.DataFrame(predictions)

    return all_years_results

In [20]:
prev_year_result = get_country_years_pred(data, "Slovakia")
prev_year_result

[1.         0.         0.02151262 0.02055928 0.02145767]
[1.         0.         0.02531902 0.02151262 0.02055928]
[1.         0.         0.03038533 0.02531902 0.02151262]
[1.         0.         0.03421163 0.03038533 0.02531902]
[1.         0.         0.03961017 0.03421163 0.03038533]
[1.         1.         0.04460578 0.03961017 0.03421163]
[1.         1.         0.01920615 0.04460578 0.03961017]
[1.         0.         0.02394134 0.01920615 0.04460578]
[1.         0.         0.0415977  0.02394134 0.01920615]


,Country,Year,Actual_Tourism,Predicted_Tourism
0,Slovakia,2015,0.025319,0.029939
1,Slovakia,2016,0.030385,0.031425
2,Slovakia,2017,0.034212,0.035546
3,Slovakia,2018,0.039610,0.041058
4,Slovakia,2019,0.044606,0.047150
5,Slovakia,2020,0.019206,0.024017
6,Slovakia,2021,0.023941,0.019130
7,Slovakia,2022,0.041598,0.040566
8,Slovakia,2023,0.048077,0.038123


In [21]:
r2_score(prev_year_result['Actual_Tourism'], prev_year_result['Predicted_Tourism'])

0.7782708312971978

In [79]:
def get_country_prediction(data, country, lag=3):
    country_df = data[data['Country'] == country].copy()
    country_df, y = get_lag_columns(country_df, lag=lag)

    lag_cols = [f'lag{i}' for i in range(1, lag + 1)]
    feature_cols = ['Bias', 'Covid'] + lag_cols

    X = country_df[feature_cols].values

    beta_tourism = fit_regression(X, y)

    last_year = country_df['Year'].max()
    #print(last_year)

    last_row = country_df[country_df['Year'] == last_year].iloc[0]
    lag_values = [last_row[f'lag{i}'] for i in range(1, lag + 1)]

    tmp_x = last_row[feature_cols].to_numpy()
    x_old = np.hstack([tmp_x[:2], y[-1], tmp_x[2:4]])
    
    year_list = [2024]
    country_list = [country]
    pred_list = [predict_next_year(beta_tourism, x_old)]

    for i in range(1, lag):
        year = last_year + i + 1
        
        X_new = np.hstack([x_old[:2], pred_list[i-1], x_old[2:4]])
        pred_tourism = predict_next_year(beta_tourism, X_new)

        year_list.append(year)
        country_list.append(country)
        pred_list.append(pred_tourism)

        x_old = X_new

    predictions = {'Year': year_list,
                   'Country': country_list,
                   'Predicted_Tourism': pred_list}

    return predictions


In [80]:
result = get_country_prediction(data, "Slovakia", lag=3)
result = pd.DataFrame(result)
result.sort_values(by=['Year'])
result

,Year,Country,Predicted_Tourism
0,2024,Slovakia,0.050925
1,2025,Slovakia,0.062911
2,2026,Slovakia,0.071675


In [81]:
def visualization(data, st_data): 
    min_trips = data['NumTrips'].min()
    max_trips = data['NumTrips'].max()

    st_data["Predicted_Tourism_normal"] = st_data["Predicted_Tourism"] * (max_trips - min_trips) + min_trips

    
    country_name = st_data["Country"].iloc[0]
    matching_data = data[data["Country"] == country_name]

    merged = pd.concat([st_data, matching_data], ignore_index=True)
    merged = merged.sort_values(by="Year").reset_index(drop=True)

    merged['Tourism_combined'] = merged['NumTrips'].fillna(merged['Predicted_Tourism_normal'])
    merged['Tourism_combined'] = merged['NumTrips'].fillna(merged['Predicted_Tourism_normal'])

    merged['Tourism_combined'] = merged['NumTrips'].fillna(merged['Predicted_Tourism_normal'])
    
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=merged['Year'], 
        y=merged['Tourism_combined'], 
        name='Tourist Numbers',
        line=dict(color='royalblue', width=4),
    ))

    fig.add_vline(
        x=2023,  
        line=dict(color='gray', dash='dot'),
        annotation_text='Prediction',
        annotation_position='top right'
)

    fig.show()


In [82]:
visualization(data_not_st, result)

In [ ]:
# create an empty df to store the predictions for all countries (call it pred_df or something)

# loop through all the countries
# for each country
# drop the last three years from data (call this subset_data or somethihg)
# temp_df = get_country_prediction(subset_data, country)
# concantenate temp_df to the bottom of pred_df
# when you are done, compare the pred_df predictions to a vector y_true
# where y_true are the 2021-2023 y values for the countries

In [77]:
def evaluate_predictions(pred_df):
    r2_tourism = r2_score(pred_df['Actual_Tourism'], pred_df['Predicted_Tourism'])
    mse_tourism = mean_squared_error(pred_df['Actual_Tourism'], pred_df['Predicted_Tourism'])

    evaluations = {
        'Tourism': {'R2': r2_tourism, 'MSE': mse_tourism}
    }

    return evaluations

In [78]:
EU_countries_list = [
    'Austria', "Belgium", "Bulgaria", "Croatia", "Czechia",
    "Denmark", "Estonia", "Finland", "France", "Germany", "Greece", "Hungary",
    "Ireland", "Italy", "Latvia", "Lithuania", "Luxembourg", "Malta",
    "Netherlands", "Poland", "Portugal", "Romania", "Slovakia", "Slovenia",
    "Spain", "Sweden"]

for country in EU_countries_list:
    country_df = data[data['Country'] == country].copy()
    # country_df

    pred_df = cross_validate_time_series(country_df, lag= 3)
    metrics = evaluate_predictions(pred_df)

    print(pred_df)
    print(country)
    print(metrics)

   Year  Actual_Tourism  Predicted_Tourism
0  2018        0.075837           0.163713
1  2019        0.079434           0.120721
2  2020        0.053121           0.082415
3  2021        0.072659           0.082217
4  2022        0.117416           0.077004
Austria
{'Tourism': {'R2': -4.475050525939516, 'MSE': 0.00240190635918188}}
   Year  Actual_Tourism  Predicted_Tourism
0  2018        0.057717          -0.284231
1  2019        0.062818           0.061475
2  2020        0.042233           0.058874
3  2021        0.057486           0.085875
4  2022        0.086910           0.080703
Belgium
{'Tourism': {'R2': -111.53917625184963, 'MSE': 0.023610275858963522}}
   Year  Actual_Tourism  Predicted_Tourism
0  2018        0.017348           0.044634
1  2019        0.016776           0.007642
2  2020        0.008258           0.081029
3  2021        0.010693           0.006796
4  2022        0.016044           0.011294
Bulgaria
{'Tourism': {'R2': -91.17016584253369, 'MSE': 0.001232269367294

LinAlgError: Singular matrix

In [ ]:
X = country_df[['Bias', 'Tourism_lag']].values
y_tourism = country_df['Tourism St.'].values
y_tourism

TypeError: tuple indices must be integers or slices, not list